# Morphological Analysis
内容：形態素解析の実行<br>
MeCabとsentencepieceを両方やる<br>
実行環境: google Colab

データの取得

In [0]:
import pandas as pd
df = pd.read_csv('/content/drive/My Drive/Colaboratory_file/NLP/neko_preprocessed.csv')
df.head() # 確認

,raw,preprocessed
0,吾輩《わがはい》は猫である。名前はまだ無い。,吾輩は猫である。名前はまだ無い。
1,どこで生れたかとんと見当《けんとう》がつかぬ。何でも薄暗いじめじめした所でニャーニャー泣い...,どこで生れたかとんと見当がつかぬ。何でも薄暗いじめじめした所でニャーニャー泣いていた事だけ...
2,この書生の掌の裏《うち》でしばらくはよい心持に坐っておったが、しばらくすると非常な速力で運...,この書生の掌の裏でしばらくはよい心持に坐っておったが、しばらくすると非常な速力で運転し始め...
3,ふと気が付いて見ると書生はいない。たくさんおった兄弟が一｜疋《ぴき》も見えぬ。肝心《かんじ...,ふと気が付いて見ると書生はいない。たくさんおった兄弟が一疋も見えぬ。肝心の母親さえ姿を隠し...
4,ようやくの思いで笹原を這い出すと向うに大きな池がある。吾輩は池の前に坐ってどうしたらよかろ...,ようやくの思いで笹原を這い出すと向うに大きな池がある。吾輩は池の前に坐ってどうしたらよかろ...


# MeCab

In [0]:
!apt-get -q -y install sudo file mecab libmecab-dev mecab-ipadic-utf8 git curl python-mecab > /dev/null
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git > /dev/null 
!echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n > /dev/null 2>&1
!pip install mecab-python3==0.996.5 > /dev/null
!echo mecab-config --dicdir"/mecab-ipadic-neologd"

Cloning into 'mecab-ipadic-neologd'...
remote: Enumerating objects: 75, done.
remote: Counting objects: 100% (75/75), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 75 (delta 5), reused 54 (delta 0), pack-reused 0
Unpacking objects: 100% (75/75), done.
mecab-config --dicdir/mecab-ipadic-neologd


In [0]:
import MeCab
m = MeCab.Tagger('-Owakati')
wakati = m.parse('COVID-19のせいで就職氷河期になりそうでぴえんって感じ')
print(wakati) # COVID-19, 就職氷河期, ぴえん を1単語とみなせていない

COVID - 19 の せい で 就職 氷河期 に なり そう で ぴえんって 感じ 



In [0]:
m = MeCab.Tagger('-Owakati -d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd')
wakati = m.parse('COVID-19のせいで就職氷河期になりそうでぴえんって感じ')
print(wakati) # ちゃんとNEologdが使えていることを確認

COVID-19 の せい で 就職氷河期 に なり そう で ぴえん って 感じ 



形態素解析の関数定義

In [0]:
def mecab_tokenizer(text):
    m = MeCab.Tagger('d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd')
    node = m.parseToNode(text)
    wordtype_list = ['名詞','動詞','形容詞']
    subtype_list = ['数', '非自立', '代名詞','接尾']
    output = []
    while node:
        if node.surface != '':
            wordtype = node.feature.split(',')[0]
            subtype = node.feature.split(',')[1]
            original = node.feature.split(',')[6]
            if wordtype in wordtype_list and subtype not in subtype_list and original != '*':
                output.append(original)
        node = node.next
        if node is None:
            break
    return output

分かち書きの実行(MeCab)

In [0]:
df['wakati_mecab'] = df['preprocessed'].apply(lambda x: ' '.join(mecab_tokenizer(x)))

In [0]:
df.head() # 確認

,raw,preprocessed,wakati_mecab
0,吾輩《わがはい》は猫である。名前はまだ無い。,吾輩は猫である。名前はまだ無い。,猫 名前 無い
1,どこで生れたかとんと見当《けんとう》がつかぬ。何でも薄暗いじめじめした所でニャーニャー泣い...,どこで生れたかとんと見当がつかぬ。何でも薄暗いじめじめした所でニャーニャー泣いていた事だけ...,生れる 見当 つく 薄暗い する 泣く いた事 記憶 する 始める 人間 見る あと 聞く ...
2,この書生の掌の裏《うち》でしばらくはよい心持に坐っておったが、しばらくすると非常な速力で運...,この書生の掌の裏でしばらくはよい心持に坐っておったが、しばらくすると非常な速力で運転し始め...,書生 掌 裏 よい 心持 坐る おる する 非常 速力 運転 する 書生 動く 自分 動く ...
3,ふと気が付いて見ると書生はいない。たくさんおった兄弟が一｜疋《ぴき》も見えぬ。肝心《かんじ...,ふと気が付いて見ると書生はいない。たくさんおった兄弟が一疋も見えぬ。肝心の母親さえ姿を隠し...,気 付く 見る 書生 いる たくさん おる 兄弟 一疋 見える 肝心 母親 姿 隠す 上今 ...
4,ようやくの思いで笹原を這い出すと向うに大きな池がある。吾輩は池の前に坐ってどうしたらよかろ...,ようやくの思いで笹原を這い出すと向うに大きな池がある。吾輩は池の前に坐ってどうしたらよかろ...,思い 笹原 這い出す 向う 池 ある 池 前 坐る する よい 考える 見る 分別 出る す...


# sentencepiece

In [0]:
!pip install sentencepiece 
import sentencepiece as spm

     |████████████████████████████████| 1.0MB 3.2MB/s 


sentencepieceがtxtからの読み込みしかできなさそうなので，いったんtxtに起こす

In [0]:
# cp932だと上手くいかなかったのでutf-8
with open('train.txt', mode='wt', encoding='utf-8') as f:
    for index, row in df.iterrows():
        if index < 5:
            print(row.preprocessed)
        f.write(row.preprocessed + '\n')

　吾輩は猫である。名前はまだ無い。
　どこで生れたかとんと見当がつかぬ。何でも薄暗いじめじめした所でニャーニャー泣いていた事だけは記憶している。吾輩はここで始めて人間というものを見た。しかもあとで聞くとそれは書生という人間中で一番獰悪な種族であったそうだ。この書生というのは時々我々を捕えて煮て食うという話である。しかしその当時は何という考もなかったから別段恐しいとも思わなかった。ただ彼の掌に載せられてスーと持ち上げられた時何だかフワフワした感じがあったばかりである。掌の上で少し落ちついて書生の顔を見たのがいわゆる人間というものの見始であろう。この時妙なものだと思った感じが今でも残っている。第一毛をもって装飾されべきはずの顔がつるつるしてまるで薬缶だ。その後猫にもだいぶ逢ったがこんな片輪には一度も出会わした事がない。のみならず顔の真中があまりに突起している。そうしてその穴の中から時々ぷうぷうと煙を吹く。どうも咽せぽくて実に弱った。これが人間の飲む煙草というものである事はようやくこの頃知った。
　この書生の掌の裏でしばらくはよい心持に坐っておったが、しばらくすると非常な速力で運転し始めた。書生が動くのか自分だけが動くのか分らないが無暗に眼が廻る。胸が悪くなる。到底助からないと思っていると、どさりと音がして眼から火が出た。それまでは記憶しているがあとは何の事やらいくら考え出そうとしても分らない。
　ふと気が付いて見ると書生はいない。たくさんおった兄弟が一疋も見えぬ。肝心の母親さえ姿を隠してしまった。その上今までの所とは違って無暗に明るい。眼を明いていられぬくらいだ。はてな何でも容子がおかしいと、のそのそ這い出して見ると非常に痛い。吾輩は藁の上から急に笹原の中へ棄てられたのである。
　ようやくの思いで笹原を這い出すと向うに大きな池がある。吾輩は池の前に坐ってどうしたらよかろうと考えて見た。別にこれという分別も出ない。しばらくして泣いたら書生がまた迎に来てくれるかと考え付いた。ニャー、ニャーと試みにやって見たが誰も来ない。そのうち池の上をさらさらと風が渡って日が暮れかかる。腹が非常に減って来た。泣きたくても声が出ない。仕方がない、何でもよいから食物のある所まであるこうと決心をしてそろりそろりと池を左りに廻り始めた。どうも非常に苦しい。そこを我慢して無理やりに這って行くとようや

In [0]:
with open('train.txt', mode='rt', encoding='utf-8') as f:
    for i, line in enumerate(f):
        if i < 5:
            print(line.strip()) # 文字化けしてない事を確認

吾輩は猫である。名前はまだ無い。
どこで生れたかとんと見当がつかぬ。何でも薄暗いじめじめした所でニャーニャー泣いていた事だけは記憶している。吾輩はここで始めて人間というものを見た。しかもあとで聞くとそれは書生という人間中で一番獰悪な種族であったそうだ。この書生というのは時々我々を捕えて煮て食うという話である。しかしその当時は何という考もなかったから別段恐しいとも思わなかった。ただ彼の掌に載せられてスーと持ち上げられた時何だかフワフワした感じがあったばかりである。掌の上で少し落ちついて書生の顔を見たのがいわゆる人間というものの見始であろう。この時妙なものだと思った感じが今でも残っている。第一毛をもって装飾されべきはずの顔がつるつるしてまるで薬缶だ。その後猫にもだいぶ逢ったがこんな片輪には一度も出会わした事がない。のみならず顔の真中があまりに突起している。そうしてその穴の中から時々ぷうぷうと煙を吹く。どうも咽せぽくて実に弱った。これが人間の飲む煙草というものである事はようやくこの頃知った。
この書生の掌の裏でしばらくはよい心持に坐っておったが、しばらくすると非常な速力で運転し始めた。書生が動くのか自分だけが動くのか分らないが無暗に眼が廻る。胸が悪くなる。到底助からないと思っていると、どさりと音がして眼から火が出た。それまでは記憶しているがあとは何の事やらいくら考え出そうとしても分らない。
ふと気が付いて見ると書生はいない。たくさんおった兄弟が一疋も見えぬ。肝心の母親さえ姿を隠してしまった。その上今までの所とは違って無暗に明るい。眼を明いていられぬくらいだ。はてな何でも容子がおかしいと、のそのそ這い出して見ると非常に痛い。吾輩は藁の上から急に笹原の中へ棄てられたのである。
ようやくの思いで笹原を這い出すと向うに大きな池がある。吾輩は池の前に坐ってどうしたらよかろうと考えて見た。別にこれという分別も出ない。しばらくして泣いたら書生がまた迎に来てくれるかと考え付いた。ニャー、ニャーと試みにやって見たが誰も来ない。そのうち池の上をさらさらと風が渡って日が暮れかかる。腹が非常に減って来た。泣きたくても声が出ない。仕方がない、何でもよいから食物のある所まであるこうと決心をしてそろりそろりと池を左りに廻り始めた。どうも非常に苦しい。そこを我慢して無理やりに這って行くとようやくの事で何

In [0]:
spm.SentencePieceTrainer.Train('--input=train.txt --model_prefix=trained_model --vocab_size=8000')
sp = spm.SentencePieceProcessor()
sp.Load('trained_model.model')

True

形態素解析の関数定義

In [0]:
import re
def spm_torkenizer(text):
    text = text.replace(' ',  '')
    text = text.replace('\n', '')
    text = text.replace(',', '')
    text = text.replace('，', '') 
    splitted_list=sp.EncodeAsPieces(text)
    output = [re.sub('▁', '', word) for word in splitted_list]
    return output

In [0]:
df['wakati_sp'] = df['preprocessed'].apply(lambda x:' '.join(spm_torkenizer(x)))

In [0]:
df.head() # 確認
# MeCabと異なり，品詞の選択はできない

,raw,preprocessed,wakati_mecab,wakati_sp
0,吾輩《わがはい》は猫である。名前はまだ無い。,吾輩は猫である。名前はまだ無い。,猫 名前 無い,吾輩は猫である 。 名前はまだ 無い 。
1,どこで生れたかとんと見当《けんとう》がつかぬ。何でも薄暗いじめじめした所でニャーニャー泣い...,どこで生れたかとんと見当がつかぬ。何でも薄暗いじめじめした所でニャーニャー泣いていた事だけ...,生れる 見当 つく 薄暗い する 泣く いた事 記憶 する 始める 人間 見る あと 聞く ...,どこ で 生れ た かと んと 見当 が つか ぬ 。 何でも 薄暗 い じ め じ め ...
2,この書生の掌の裏《うち》でしばらくはよい心持に坐っておったが、しばらくすると非常な速力で運...,この書生の掌の裏でしばらくはよい心持に坐っておったが、しばらくすると非常な速力で運転し始め...,書生 掌 裏 よい 心持 坐る おる する 非常 速力 運転 する 書生 動く 自分 動く ...,この 書生 の 掌 の 裏 で しばらく は よい 心持 に 坐って おった が 、 しば...
3,ふと気が付いて見ると書生はいない。たくさんおった兄弟が一｜疋《ぴき》も見えぬ。肝心《かんじ...,ふと気が付いて見ると書生はいない。たくさんおった兄弟が一疋も見えぬ。肝心の母親さえ姿を隠し...,気 付く 見る 書生 いる たくさん おる 兄弟 一疋 見える 肝心 母親 姿 隠す 上今 ...,ふと 気が 付いて 見ると 書生 は いない 。 た く さん おった 兄 弟 が一 疋 ...
4,ようやくの思いで笹原を這い出すと向うに大きな池がある。吾輩は池の前に坐ってどうしたらよかろ...,ようやくの思いで笹原を這い出すと向うに大きな池がある。吾輩は池の前に坐ってどうしたらよかろ...,思い 笹原 這い出す 向う 池 ある 池 前 坐る する よい 考える 見る 分別 出る す...,ようやく の 思い で 笹原 を 這 い 出すと 向う に 大きな 池 がある 。 吾輩は...


# 保存

In [0]:
df.to_csv('/content/drive/My Drive/Colaboratory_file/NLP/neko_wakati.csv', header=True, index=False)

In [0]:
test_df = pd.read_csv('/content/drive/My Drive/Colaboratory_file/NLP/neko_wakati.csv')
test_df.head() # 確認

,raw,preprocessed,wakati_mecab,wakati_sp
0,吾輩《わがはい》は猫である。名前はまだ無い。,吾輩は猫である。名前はまだ無い。,猫 名前 無い,吾輩は猫である 。 名前はまだ 無い 。
1,どこで生れたかとんと見当《けんとう》がつかぬ。何でも薄暗いじめじめした所でニャーニャー泣い...,どこで生れたかとんと見当がつかぬ。何でも薄暗いじめじめした所でニャーニャー泣いていた事だけ...,生れる 見当 つく 薄暗い する 泣く いた事 記憶 する 始める 人間 見る あと 聞く ...,どこ で 生れ た かと んと 見当 が つか ぬ 。 何でも 薄暗 い じ め じ め ...
2,この書生の掌の裏《うち》でしばらくはよい心持に坐っておったが、しばらくすると非常な速力で運...,この書生の掌の裏でしばらくはよい心持に坐っておったが、しばらくすると非常な速力で運転し始め...,書生 掌 裏 よい 心持 坐る おる する 非常 速力 運転 する 書生 動く 自分 動く ...,この 書生 の 掌 の 裏 で しばらく は よい 心持 に 坐って おった が 、 しば...
3,ふと気が付いて見ると書生はいない。たくさんおった兄弟が一｜疋《ぴき》も見えぬ。肝心《かんじ...,ふと気が付いて見ると書生はいない。たくさんおった兄弟が一疋も見えぬ。肝心の母親さえ姿を隠し...,気 付く 見る 書生 いる たくさん おる 兄弟 一疋 見える 肝心 母親 姿 隠す 上今 ...,ふと 気が 付いて 見ると 書生 は いない 。 た く さん おった 兄 弟 が一 疋 ...
4,ようやくの思いで笹原を這い出すと向うに大きな池がある。吾輩は池の前に坐ってどうしたらよかろ...,ようやくの思いで笹原を這い出すと向うに大きな池がある。吾輩は池の前に坐ってどうしたらよかろ...,思い 笹原 這い出す 向う 池 ある 池 前 坐る する よい 考える 見る 分別 出る す...,ようやく の 思い で 笹原 を 這 い 出すと 向う に 大きな 池 がある 。 吾輩は...
